In [1]:
import argparse
import math
import os
import random
import time

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
import pandas as pd

from dataset import TimeMachineData, TimeMachineDataset

In [2]:
def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [3]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=128, nlayers=2):
        super(RNN, self).__init__()
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.nlayers = nlayers

        self.embed = nn.Embedding(vocab_size, emb_dim)
        self.dropout1 = nn.Dropout(0.5)
        self.rnn = nn.RNN(emb_dim, hidden_dim, 2)
        self.dropout2 = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_dim, vocab_size)

        nn.init.normal_(self.embed.weight, std=0.01)

        nn.init.normal_(self.rnn.weight_ih_l0, std=1 / math.sqrt(emb_dim))
        nn.init.normal_(self.rnn.weight_hh_l0, std=1 / math.sqrt(emb_dim))
        nn.init.zeros_(self.rnn.bias_ih_l0)
        nn.init.zeros_(self.rnn.bias_hh_l0)

    def forward(self, inputs, hidden):
        out = self.embed(inputs)
        out = self.dropout1(out)
        out, hidden = self.rnn(out, hidden)
        out = self.dropout2(out)
        out = self.linear(out)

        return out, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.nlayers, bsz, self.hidden_dim).zero_())

In [4]:
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i : i + seq_len]
    target = source[i + 1 : i + 1 + seq_len].reshape(-1)
    return data, target

In [5]:
# device = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
temperature = 1.0
bptt = 35

In [6]:
timemachine = TimeMachineData()

In [7]:
ntokens = timemachine.vocab_size

In [8]:
model = RNN(ntokens)
model.load_state_dict(torch.load("rnn_thetimemachine.pth"))
model = model.to(device)
model.eval()

RNN(
  (embed): Embedding(94, 128)
  (dropout1): Dropout(p=0.5, inplace=False)
  (rnn): RNN(128, 128, num_layers=2)
  (dropout2): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=128, out_features=94, bias=True)
)

In [9]:
text_path="../data/TheTimeMachine/35-0.txt"
dict_path="../data/TheTimeMachine/char.csv"

In [10]:
word_df = pd.read_csv(dict_path)
char2idx = dict(zip(word_df["word"], word_df.index))
idx2char = dict(zip(word_df.index, word_df["word"]))

# multi char input

In [11]:
text = "A queer thing I soon"
text = [char2idx["<BOS>"]] + [char2idx[char] for char in text.strip()]
data = [text]
data = torch.from_numpy(np.array(sum(data, []))).long()
data = torch.unsqueeze(data, 0)

In [12]:
data

tensor([[ 0, 31,  3, 51, 15,  4,  4, 11,  3,  5, 12,  9,  8, 19,  3, 26,  3, 10,
          7,  7,  8]])

In [13]:
data = data.to(device)

In [14]:
data

tensor([[ 0, 31,  3, 51, 15,  4,  4, 11,  3,  5, 12,  9,  8, 19,  3, 26,  3, 10,
          7,  7,  8]])

In [15]:
hidden = model.init_hidden(data.shape[1])
result = data.cpu().numpy().tolist()[0]
for i in range(100):
    with torch.no_grad():
        output, hidden = model(data, hidden)
        
        word_weights = output.squeeze().data.div(1.0).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1).squeeze()
        
        result += word_idx.cpu().numpy().tolist()
        data = word_idx.reshape(1, -1)
        
        if 0 in word_idx.cpu().numpy().tolist():
            break

In [16]:
"".join([idx2char[x] for x in result])

'<BOS>A queer thing I soonavTun- kt/enaoS<EOS>Y 6kXnihkddwehem tup<BOS>it. /'

# single char input

In [17]:
for j in range(10):
    text = [0]
    data = [text]
    data = torch.from_numpy(np.array(sum(data, []))).long()
    data = torch.unsqueeze(data, 0)
    data = data.to(device)
    
    hidden = model.init_hidden(1)
    result = [0]
    for i in range(100):
        output, hidden = model(data, hidden)
        
        # word_idx = torch.argmax(output, dim=2).item()
        word_weights = output.squeeze().data.div(1.0).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        
        data.data.fill_(word_idx)
        result.append(word_idx.item())
        if word_idx == 1:
            break
    print("".join([idx2char[x] for x in result]))

<BOS>could. The Daggh, Ge as<EOS>
<BOS>aud see1l. Sut’cifh’illiches, and in the rooldend, and dard—a was diman (bonay :ape. I have no to me
<BOS>intomes, por and ito<EOS>
<BOS>in the Mdevers think, aüsuvated to ag. The<EOS>
<BOS>hand us sole#tress foscals a gent made. Wo the Way<EOS>
<BOS>That I stoidging enough me heexe<EOS>
<BOS>the<EOS>
<BOS>of all the minder, hud mrying latheors, I had wother and douss hit sour. bat they was reworal be. Th
<BOS>“foon I wan indeld ot and htatchess ser more tto dist. Hade as riwrle littre heey un<EOS>
<BOS>they; I gotæ wele grave the<EOS>
